In [4]:
import os
from dataclasses import dataclass
from pathlib import Path
from typing import List

In [2]:
os.chdir("..")

In [16]:
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    training_data_path: Path
    pretrained_model_path: Path
    model_filename: str
    augmentation: bool
    batch_size: int
    epochs: int
    image_size: List

In [17]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories

In [18]:
class ConfigManager:

    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
    
    def get_training_config(self) -> TrainingConfig:
        config = self.config.training

        training_config = TrainingConfig(
            root_dir=config.root_dir,
            training_data_path=config.training_data_path,
            pretrained_model_path=config.pretrained_model_path,
            model_filename=config.model_filename,
            augmentation=self.params.AUGMENTATION,
            batch_size=self.params.BATCH_SIZE,
            epochs=self.params.EPOCHS,
            image_size=self.params.IMAGE_SIZE
        )

        return training_config

In [19]:
import os
import tensorflow as tf
from cnnClassifier.exception import CustomException
from cnnClassifier.logger import logging

In [26]:
class Training:

    def __init__(self, config: TrainingConfig):
        self.config = config
        self.model = None
        self.train_generator = None
        self.valid_generator = None
        self.get_base_model()

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.pretrained_model_path)
    
    @staticmethod
    def save_model(model: tf.keras.Model, path: Path):
        model.save(path)
    
    def _train_valid_generator(self):

        data_generator_kwargs = dict(
            rescale=1./255,
            validation_split = 0.2
        )

        valid_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(**data_generator_kwargs)

        dataflow_kwargs = dict(
            target_size = self.config.image_size[:-1],
            batch_size = self.config.batch_size,
            interpolation="bilinear"
        )

        self.valid_generator = valid_data_generator.flow_from_directory(
            self.config.training_data_path,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.augmentation:
            train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **data_generator_kwargs
            )
        else:
            train_data_generator = valid_data_generator
        
        self.train_generator = train_data_generator.flow_from_directory(
            directory=self.config.training_data_path,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
    
    def train(self):
        train_steps = self.train_generator.samples // self.train_generator.batch_size
        validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs = self.config.epochs,
            steps_per_epoch = train_steps,
            validation_data=self.valid_generator,
            validation_steps=validation_steps
        )
        path = os.path.join(self.config.root_dir, self.config.model_filename)

        self.save_model(model=self.model, path=path)


    


In [27]:
try:
    config = ConfigManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training._train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
Epoch 1/5
17/17 [==============================] - 30s 2s/step - loss: 0.8568 - accuracy: 0.5328 - val_loss: 0.6914 - val_accuracy: 0.6094
Epoch 2/5
17/17 [==============================] - 29s 2s/step - loss: 0.6912 - accuracy: 0.5753 - val_loss: 0.6891 - val_accuracy: 0.6094
Epoch 3/5
17/17 [==============================] - 28s 2s/step - loss: 0.6908 - accuracy: 0.5637 - val_loss: 0.6879 - val_accuracy: 0.6094
Epoch 4/5
17/17 [==============================] - 30s 2s/step - loss: 0.6899 - accuracy: 0.5625 - val_loss: 0.6872 - val_accuracy: 0.6094
Epoch 5/5
17/17 [==============================] - 28s 2s/step - loss: 0.6899 - accuracy: 0.5560 - val_loss: 0.6854 - val_accuracy: 0.6094


/Users/ngkuissi/miniforge3/envs/chest/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
